<a href="https://colab.research.google.com/github/maxhormazabal/depencendy_parsing/blob/main/p2_ann_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Experimentation

In [1]:
!pip install conllu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Getting access to Google Drive files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
os.chdir("/content/drive/MyDrive/MASTER")
from nlu_model_utils import *
from nlu_preprocessing_utils import *
import tensorflow as tf
from tensorflow import keras
from keras import Input, Model
from keras.layers import Dense
import pandas as pd
import numpy as np
import math
import time
import json

In [4]:
# English
# 'https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/master/en_ewt-ud-train.conllu'
# https://raw.githubusercontent.com/UniversalDependencies/UD_English-ParTUT/master/en_partut-ud-train.conllu

base_url = 'https://raw.githubusercontent.com/UniversalDependencies/UD_English-ParTUT/master/'
file_basename = 'en_partut-ud'
(en_train,en_test,en_val) = readConlluDataset(base_url,file_basename)
en_upo2number, en_number2upo, en_nupos = getUposList(en_train)
number2action,action2number = getActionDict()
number2deprelTest = getNumber2Deprel()

In [5]:
train_df = conlluToDatasetForDependency(en_train,en_upo2number)
test_df = conlluToDatasetForDependency(en_test,en_upo2number)
val_df = conlluToDatasetForDependency(en_val,en_upo2number)

In [6]:
stack_len = 3
buffer_len = 5

path = "nlu_data/"+str(stack_len)+"stack"+str(buffer_len)+"buffer"

(x_train_stack,x_train_buffer,x_train_stack_upos,x_train_buffer_upos,action_train,deprel_train) = getDataSource('train',folder=path)
(x_test_stack,x_test_buffer,x_test_stack_upos,x_test_buffer_upos,action_test,deprel_test) = getDataSource('test',folder=path)
(x_val_stack,x_val_buffer,x_val_stack_upos,x_val_buffer_upos,action_val,deprel_val) = getDataSource('val',folder=path)

(84682, 3) (84682, 5) (84682, 3) (84682, 5) (84682, 4) (84682, 44)
(6702, 3) (6702, 5) (6702, 3) (6702, 5) (6702, 4) (6702, 44)
(5282, 3) (5282, 5) (5282, 3) (5282, 5) (5282, 4) (5282, 44)


In [7]:
model = buildModelA(stack_len,buffer_len,action_train[0].shape[0],deprel_train[0].shape[0],
                    optimizer='adam',learning_rate=0.1,
                    embedd_output=50,loss='categorical_crossentropy')

callback = tf.keras.callbacks.EarlyStopping(monitor='val_Deprel_Output_accuracy', patience=5,restore_best_weights=True)

model.fit([x_train_stack,x_train_buffer],
          [action_train, deprel_train],
          epochs=15, batch_size=512,
          callbacks=[callback],
          validation_data=([x_val_stack,x_val_buffer],[action_val,deprel_val]))
score = model.evaluate([x_test_stack,x_test_buffer],[action_test, deprel_test])

Epoch 1/15
166/166 [==============================] - 24s 87ms/step - loss: 2.8383 - Action_Output_loss: 1.1444 - Deprel_Output_loss: 1.6939 - Action_Output_accuracy: 0.4742 - Deprel_Output_accuracy: 0.5996 - val_loss: 3.3949 - val_Action_Output_loss: 1.0063 - val_Deprel_Output_loss: 2.3886 - val_Action_Output_accuracy: 0.5293 - val_Deprel_Output_accuracy: 0.5449
Epoch 2/15
166/166 [==============================] - 11s 65ms/step - loss: 2.4881 - Action_Output_loss: 1.0249 - Deprel_Output_loss: 1.4632 - Action_Output_accuracy: 0.5252 - Deprel_Output_accuracy: 0.5993 - val_loss: 3.6071 - val_Action_Output_loss: 1.0090 - val_Deprel_Output_loss: 2.5980 - val_Action_Output_accuracy: 0.5222 - val_Deprel_Output_accuracy: 0.5710
Epoch 3/15
166/166 [==============================] - 11s 63ms/step - loss: 2.4513 - Action_Output_loss: 1.0114 - Deprel_Output_loss: 1.4399 - Action_Output_accuracy: 0.5273 - Deprel_Output_accuracy: 0.6006 - val_loss: 3.8218 - val_Action_Output_loss: 1.0259 - val_Dep

In [8]:
# Guardar el Modelo
model.save('models/testing_model.h5')

In [7]:
# Recrea exactamente el mismo modelo solo desde el archivo
model = keras.models.load_model('models/testing_model.h5')

In [8]:
(tokenizer,n_token) = getTokenizer('nlu_data')

Tokenizer with  6303  word sucessfully obtained.


In [ ]:
(head,deprel) = testingPredictions(3,5,test_df.iloc[1,:],tokenizer,model,number2deprelTest)

In [9]:
# Check lengths of en_test and test_df
# stack_len = 3
# buffer_len = 5

base_url = 'https://raw.githubusercontent.com/UniversalDependencies/UD_English-ParTUT/master/'
file_basename = 'en_partut-ud'
(en_train,en_test,en_val) = readConlluDataset(base_url,file_basename)

test_df = conlluToDatasetForDependency(en_test,en_upo2number)

# Delete UPOS = _
for (i,token_list) in enumerate(en_test):
  for (j,dictionary) in enumerate(token_list):
    if dictionary['upos'] == "_":
      en_test[i][j].clear()

list_projective = projectiveArcs(test_df)

# Deleting non-projective
for (i,token_list) in enumerate(en_test):
  if i not in list_projective:
    token_list.clear()

# Iterate each row of df_test, and take the head en deprel.
for row in range(len(test_df)):
  (head,deprel) = testingPredictions(stack_len,buffer_len,test_df.iloc[row,:],tokenizer,model,number2deprelTest)
  for word in range(len(en_test[row])):
    if len(en_test[row]) > len(head):
      head = np.concatenate((head,np.full(len(en_test[row])-len(head),int(0))))
    if len(en_test[row]) > len(deprel):
      deprel = np.concatenate((deprel,np.full(len(en_test[row])-len(deprel),int(0))))
    en_test[row][word]['head'] = head[word]
    en_test[row][word]['deprel'] = deprel[word]

1
2
3
4
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
1
2
3
4
5
6
7
8
9
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
1
2
3
4
5
6
7
8
9
10
11
12
13
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56

In [10]:
import io

!rm nlu_data/predicted_test.conllu

with io.open("nlu_data/predicted_test.conllu", mode='a', encoding='utf-8') as f:
  for token_list in en_test:
    serialized = token_list.serialize()
    f.write(serialized)
print('Predicted file generated in nlu_data/predicted_test.conllu')

with open('nlu_data/predicted_test.conllu', 'r') as f:
  # Leemos todo el contenido del archivo en una variable
  contenido = f.read()

lineas = contenido.split('\n\n')

lineas_sin_vacias = []

# Recorremos la lista de líneas
for linea in lineas:
  # Si la línea no es vacía, la añadimos a la lista
  if linea.strip():
    lineas_sin_vacias.append(linea)

contenido_con_un_salto_linea = '\n\n'.join(lineas_sin_vacias)

with open('nlu_data/predicted_test_line.conllu', 'w') as f:
  # Escribimos el contenido con solo un salto de línea en el archivo
  f.write(contenido_con_un_salto_linea)

Predicted file generated in nlu_data/predicted_test.conllu


In [ ]:
head,deprel = testingPredictions(stack_len,buffer_len,test_df.iloc[26,:],tokenizer,model,number2deprelTest)

In [36]:
!cd nlu_data;python 'conll18_ud_eval.py' 'original_test_line.conllu' 'predicted_test_line.conllu' -v

Traceback (most recent call last):
  File "conll18_ud_eval.py", line 536, in <module>
    main()
  File "conll18_ud_eval.py", line 504, in main
    evaluation = evaluate_wrapper(args)
  File "conll18_ud_eval.py", line 486, in evaluate_wrapper
    gold_ud = load_conllu_file(args.gold_file)
  File "conll18_ud_eval.py", line 482, in load_conllu_file
    return load_conllu(_file)
  File "conll18_ud_eval.py", line 213, in load_conllu
    process_word(word)
  File "conll18_ud_eval.py", line 201, in process_word
    raise UDError("HEAD '{}' '{}' points outside of the sentence".format(_encode(word.columns[HEAD]),word.columns))
__main__.UDError: HEAD '5' '['1', 'If', 'if', 'SCONJ', 'CS', '', '5', 'mark', '_', '_']' points outside of the sentence


In [24]:
with open('nlu_data/predicted_test_line.conllu', 'r') as f:
  # Leemos todo el contenido del archivo en una variable
  contenido = f.read()

# Contamos el número de veces que aparece el carácter de nueva línea
num_saltos_linea = contenido.count('\n\n')

# Comprobamos si el archivo tiene más de un salto de línea
if num_saltos_linea > 1:
  print("El archivo tiene más de un salto de línea.")
else:
  print("El archivo no tiene más de un salto de línea.")

num_saltos_linea

El archivo no tiene más de un salto de línea.


0

In [22]:
with open('nlu_data/predicted_test.conllu', 'r') as f:
  # Leemos todo el contenido del archivo en una variable
  contenido = f.read()

lineas = contenido.split('\n\n')

lineas_sin_vacias = []

# Recorremos la lista de líneas
for linea in lineas:
  # Si la línea no es vacía, la añadimos a la lista
  if linea.strip():
    lineas_sin_vacias.append(linea)

contenido_con_un_salto_linea = '\n\n'.join(lineas_sin_vacias)

with open('nlu_data/predicted_test_line.conllu', 'w') as f:
  # Escribimos el contenido con solo un salto de línea en el archivo
  f.write(contenido_con_un_salto_linea)

In [13]:
import random

print(random.randint(0,9))

8


In [27]:
id = [0,1,2,3,4,5,6,7]

head = [0,0,999,2,999,4,999,999]

vector = head

# Revisar si existe más de un cero:
k = 0
for i,element in enumerate(vector):
  if element == 0:
    k = k + 1

# Si existe más de uno 
if k>1:
  for i,element in enumerate(vector):
    if element == 0 and k>1:
      vector[i] = 999
      k = k - 1
for i,element in enumerate(vector):
  if element == 999 and k == 0:
    vector[i] = 0
    break
for i,element in enumerate(vector):
  if element == 999:
    for l in range(1,len(head)):
      if l not in vector:
        print(l)
        vector[i] = l
        break


vector

1
3
5
6
7


[1, 0, 3, 2, 5, 4, 6, 7]

In [31]:

original = './nlu_data/original_test.conllu'
predicted = './nlu_data/predicted_test2.conllu'

!python /content/drive/MyDrive/MASTER/nlu_data/conll18_ud_eval.py -v $original $predicted

Traceback (most recent call last):
  File "/content/drive/MyDrive/MASTER/nlu_data/conll18_ud_eval.py", line 532, in <module>
    main()
  File "/content/drive/MyDrive/MASTER/nlu_data/conll18_ud_eval.py", line 500, in main
    evaluation = evaluate_wrapper(args)
  File "/content/drive/MyDrive/MASTER/nlu_data/conll18_ud_eval.py", line 482, in evaluate_wrapper
    gold_ud = load_conllu_file(args.gold_file)
  File "/content/drive/MyDrive/MASTER/nlu_data/conll18_ud_eval.py", line 478, in load_conllu_file
    return load_conllu(_file)
  File "/content/drive/MyDrive/MASTER/nlu_data/conll18_ud_eval.py", line 209, in load_conllu
    process_word(word)
  File "/content/drive/MyDrive/MASTER/nlu_data/conll18_ud_eval.py", line 201, in process_word
    raise UDError("HEAD '{}' points outside of the sentence".format(_encode(word.columns[HEAD])))
__main__.UDError: HEAD '5' points outside of the sentence


In [28]:

original = 'nlu_data/original_test.conllu'
predicted = 'nlu_data/predicted_test2.conllu'
!cd nlu_data
!cd nlu_data;diff -y 'original_test.conllu' 'predicted_test.conllu2'

# sent_id = en_partut-ud-1					# sent_id = en_partut-ud-1
# text = Attribution-ShareAlike 2.0.				# text = Attribution-ShareAlike 2.0.
1	Attribution	attribution	NOUN	S	Numbe	1	Attribution	attribution	NOUN	S	Numbe
2	-	-	PUNCT	FF	_	1	punct |	2	-	-	PUNCT	FF	_	1	None
3	ShareAlike	ShareAlike	PROPN	SP	_     |	3	ShareAlike	ShareAlike	PROPN	SP	_
4	2.0	2.0	NUM	N	NumType=Card	3     |	4	2.0	2.0	NUM	N	NumType=Card	1
5	.	.	PUNCT	FS	_	1	punct |	5	.	.	PUNCT	FS	_	1	None

# sent_id = en_partut-ud-9					# sent_id = en_partut-ud-9
# text = Any use of the work other than as authorized under t	# text = Any use of the work other than as authorized under t
1	Any	any	DET	DI	PronType=Ind	2     |	1	Any	any	DET	DI	PronType=Ind	0
2	use	use	NOUN	S	Number=Sing	17    |	2	use	use	NOUN	S	Number=Sing	1
3	of	of	ADP	E	_	5	case  |	3	of	of	ADP	E	_	1	None
4	the	the	DET	RD	Definite=Def|PronType |	4	the	the	DET	RD	Definite=Def|PronType
5	work	work	NOUN	S	Number=Sing	2     |	5	work	work	NOUN	S	Number=Sing	1
6	other	other	ADJ	A

In [26]:
!cd nlu_data;cat original_test.conllu

# sent_id = en_partut-ud-1
# text = Attribution-ShareAlike 2.0.
1	Attribution	attribution	NOUN	S	Number=Sing	0	root	_	SpaceAfter=No
2	-	-	PUNCT	FF	_	1	punct	_	SpaceAfter=No
3	ShareAlike	ShareAlike	PROPN	SP	_	1	flat	_	_
4	2.0	2.0	NUM	N	NumType=Card	3	nummod	_	SpaceAfter=No
5	.	.	PUNCT	FS	_	1	punct	_	_

# sent_id = en_partut-ud-9
# text = Any use of the work other than as authorized under this license or copyright law is prohibited.
1	Any	any	DET	DI	PronType=Ind	2	det	_	_
2	use	use	NOUN	S	Number=Sing	17	nsubj:pass	_	_
3	of	of	ADP	E	_	5	case	_	_
4	the	the	DET	RD	Definite=Def|PronType=Art	5	det	_	_
5	work	work	NOUN	S	Number=Sing	2	nmod	_	_
6	other	other	ADJ	A	Degree=Pos	2	amod	_	_
7	than	than	SCONJ	CS	_	9	mark	_	_
8	as	as	ADP	E	_	9	mark	_	_
9	authorized	authorize	VERB	V	Tense=Past|VerbForm=Part	6	advcl	_	_
10	under	under	ADP	E	_	12	case	_	_
11	this	this	DET	DD	Number=Sing|PronType=Dem	12	det	_	_
12	license	license	NOUN	S	Number=Sing	9	obl	_	_
13	or	or	CCONJ	CC	_	15	cc	_	_
14	copyright	copy

In [ ]:
def testingPredictions(stack_len,buffer_len,df_row,tokenizer,model,number2deprelTest):

  children = []

  # re-write variables including root at the begining
  form = np.concatenate((['root'],df_row['form']))
  id = np.concatenate(([int(0)],df_row['id']))
  head = np.full(len(id),999)
  deprel = np.full(len(id),'999',dtype='object')

  stack = form[0]
  buffer = form[1:]

  # First prediction
  (action,pred_deprel) = predTestValues(stackToVector(tokenizer.texts_to_sequences(['root'])[0],3),
                bufferToVector(np.array(tokenizer.texts_to_sequences(buffer)).ravel(),5),
                children,
                model)

  #print(stack," | ",buffer," | ",action)

  while len(buffer) > 1:
    s = stack[-1] #setting attention in the last element on stack
    b = buffer[0] #setting attention in the first element on buffer

    # Left arc
    if (action == 0):
      children.append(id[findValueIndex(form,s)])
      stack = np.delete(stack,-1)
      head[findValueIndex(form,s)] = id[findValueIndex(form,b)]
      deprel[findValueIndex(form,s)] = number2deprelTest[pred_deprel]
      action_name = 'Left Arc'

    # Right arc
    elif (action == 1):  #if s is the father of b
      children.append(id[findValueIndex(form,b)])
      stack = np.append(stack,b)
      buffer = np.delete(buffer,0)
      head[findValueIndex(form,b)] = id[findValueIndex(form,s)]
      deprel[findValueIndex(form,b)] = number2deprelTest[pred_deprel]
      action_name = 'Right Arc'

    # Reduce
    elif (action == 2):
      stack = np.delete(stack,-1)
      action_name = 'Reduce'
    
    # Shift
    elif (action == 3):
      stack = np.append(stack,b)
      buffer = np.delete(buffer,0)
      action_name = 'Shift'

    #print(stack,stack.shape," | ",buffer,buffer.shape," | ",action_name)
    (action,pred_deprel) = predTestValues(stackToVector(tokenizer.texts_to_sequences(stack)[0],3),
                  bufferToVector(np.array(tokenizer.texts_to_sequences(buffer)).ravel(),5),
                  children,
                  model)
    
  head = checkHead(head[1:])
  deprel = checkDeprel(deprel[1:])
  return (head,deprel)

In [ ]:
def testingPredictions(stack_len,buffer_len,df_row,tokenizer,model,number2deprelTest):

  children = []

  # re-write variables including root at the begining
  form = np.concatenate((['root'],df_row['form']))
  id = np.concatenate(([int(0)],df_row['id']))
  head = np.full(len(id),999)
  deprel = np.full(len(id),'999',dtype='object')

  stack = form[0]
  buffer = form[1:]

  # First prediction
  (action,pred_deprel) = predTestValues(stackToVector(tokenizer.texts_to_sequences(['root'])[0],3),
                bufferToVector(tokenizer.texts_to_sequences(buffer),5),
                children,
                model)

  #print(stack," | ",buffer," | ",action)

  while len(buffer) > 1:
    s = stack[-1] #setting attention in the last element on stack
    b = buffer[0] #setting attention in the first element on buffer

    # Left arc
    if (action == 0):
      children.append(id[findValueIndex(form,s)])
      stack = np.delete(stack,-1)
      head[findValueIndex(form,s)] = id[findValueIndex(form,b)]
      deprel[findValueIndex(form,s)] = number2deprelTest[pred_deprel]
      action_name = 'Left Arc'

    # Right arc
    elif (action == 1):  #if s is the father of b
      children.append(id[findValueIndex(form,b)])
      stack = np.append(stack,b)
      buffer = np.delete(buffer,0)
      head[findValueIndex(form,b)] = id[findValueIndex(form,s)]
      deprel[findValueIndex(form,b)] = number2deprelTest[pred_deprel]
      action_name = 'Right Arc'

    # Reduce
    elif (action == 2):
      stack = np.delete(stack,-1)
      action_name = 'Reduce'
    
    # Shift
    elif (action == 3):
      stack = np.append(stack,b)
      buffer = np.delete(buffer,0)
      action_name = 'Shift'

    #print(stack,stack.shape," | ",buffer,buffer.shape," | ",action_name)
    (action,pred_deprel) = predTestValues(stackToVector(tokenizer.texts_to_sequences(stack)[0],3),
                  bufferToVector(np.array(tokenizer.texts_to_sequences(buffer)).ravel(),5),
                  children,
                  model)
    
  head = checkHead(head)
  deprel = checkDeprel(deprel)
  return (head,deprel)

In [ ]:
df_row = test_df.iloc[1,:]

# first execution
stack_len = 3
buffer_len = 5

children = []

# re-write variables including root at the begining
form = np.concatenate((['root'],df_row['form']))
id = np.concatenate(([int(0)],df_row['id']))
head = np.full(len(id),999)
deprel = np.full(len(id),'999',dtype='object')

stack = form[0]
buffer = form[1:]

# First prediction
(action,pred_deprel) = predTestValues(stackToVector(tokenizer.texts_to_sequences(['root'])[0],3),
               bufferToVector(tokenizer.texts_to_sequences(buffer),5),
               children,
               model)

print(stack," | ",buffer," | ",action)

while len(buffer) > 1:
  s = stack[-1] #setting attention in the last element on stack
  b = buffer[0] #setting attention in the first element on buffer

  # Left arc
  if (action == 0):
    children.append(id[findValueIndex(form,s)])
    stack = np.delete(stack,-1)
    head[findValueIndex(form,s)] = id[findValueIndex(form,b)]
    deprel[findValueIndex(form,s)] = number2deprelTest[pred_deprel]
    action_name = 'Left Arc'

  # Right arc
  elif (action == 1):  #if s is the father of b
    children.append(id[findValueIndex(form,b)])
    stack = np.append(stack,b)
    buffer = np.delete(buffer,0)
    head[findValueIndex(form,b)] = id[findValueIndex(form,s)]
    deprel[findValueIndex(form,b)] = number2deprelTest[pred_deprel]
    action_name = 'Right Arc'

  # Reduce
  elif (action == 2):
    stack = np.delete(stack,-1)
    action_name = 'Reduce'
  
  # Shift
  elif (action == 3):
    stack = np.append(stack,b)
    buffer = np.delete(buffer,0)
    action_name = 'Shift'

  print(stack,stack.shape," | ",buffer,buffer.shape," | ",action_name)
  (action,pred_deprel) = predTestValues(stackToVector(tokenizer.texts_to_sequences(stack)[0],3),
                bufferToVector(np.array(tokenizer.texts_to_sequences(buffer)).ravel(),5),
                children,
                model)
  
head = checkHead(head)
deprel = checkDeprel(deprel)

root  |  ['Any' 'use' 'of' 'the' 'work' 'other' 'than' 'as' 'authorized' 'under'
 'this' 'license' 'or' 'copyright' 'law' 'is' 'prohibited' '.']  |  3
['root' 'Any'] (2,)  |  ['use' 'of' 'the' 'work' 'other' 'than' 'as' 'authorized' 'under' 'this'
 'license' 'or' 'copyright' 'law' 'is' 'prohibited' '.'] (17,)  |  Shift
['root' 'Any' 'use'] (3,)  |  ['of' 'the' 'work' 'other' 'than' 'as' 'authorized' 'under' 'this'
 'license' 'or' 'copyright' 'law' 'is' 'prohibited' '.'] (16,)  |  Right Arc
['root' 'Any' 'use' 'of'] (4,)  |  ['the' 'work' 'other' 'than' 'as' 'authorized' 'under' 'this' 'license'
 'or' 'copyright' 'law' 'is' 'prohibited' '.'] (15,)  |  Shift
['root' 'Any' 'use' 'of' 'the'] (5,)  |  ['work' 'other' 'than' 'as' 'authorized' 'under' 'this' 'license' 'or'
 'copyright' 'law' 'is' 'prohibited' '.'] (14,)  |  Shift
['root' 'Any' 'use' 'of' 'the' 'work'] (6,)  |  ['other' 'than' 'as' 'authorized' 'under' 'this' 'license' 'or'
 'copyright' 'law' 'is' 'prohibited' '.'] (13,)  |  S

In [ ]:
def checkHead(vector):
  for i,element in enumerate(vector):
    if element == 999:
      vector[i] = 0
      break
  for i,element in enumerate(vector):
    if element == 999:
      vector[i] = 1
  return vector

def checkDeprel(vector):
  for i,element in enumerate(vector):
    if element == '999':
      vector[i] = 'root'
      break
  for i,element in enumerate(vector):
    if element == '999':
      vector[i] = 'None'
  return vector

In [ ]:
checkDeprel(deprel)

array(['root', 'None', 'None', 'None', 'None', 'None', 'None', 'None',
       'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None',
       'None', 'None', 'None'], dtype=object)

In [ ]:
bufferToVector(np.array(tokenizer.texts_to_sequences(buffer)).ravel(),5)

array([178, 11, 3950, 4, 0], dtype=object)

In [ ]:
stack = tokenizer.texts_to_sequences(['root'])[0]
stackToVector(tokenizer.texts_to_sequences(['root'])[0],4)

array(['Any', 'use', 'of', 'the', 'work', 'other', 'than', 'as',
       'authorized', 'under', 'this', 'license', 'or', 'copyright', 'law',
       'is', 'prohibited', '.'], dtype=object)

In [ ]:
df_row = test_df.iloc[1,:]

# first execution
stack_len = 3
buffer_len = 5

children = []

stack = x_test_stack[0]
buffer = x_test_buffer[0]

#(action,children) = predNewValues(stack_len,buffer_len,'root',en_train[0].metadata['text'],children,model)
(action,pred_deprel) = predTestValues(stack,buffer,children,model)

id_stack = [0]
id_buffer = [*range(1, len(buffer)+1)]

# re-write variables including root at the begining
form = np.concatenate((['root'],df_row['form']))
id = np.concatenate(([int(0)],df_row['id']))
head = np.full(len(id),999)
deprel = np.full(len(id),999)

stack_word = form[0]
buffer_word = form[1:]

print(stack_word," | ",buffer_word," | ",action)

i = 0
while len(buffer) > 1:
  s = stack[-1] #setting attention in the last element on stack
  b = buffer[0] #setting attention in the first element on buffer

  s_word = stack_word[-1]
  b_word = buffer_word[0]

  #Checking Right Arc
  # finValueIndex search the text of the current value and it returns the position
  
  # Left arc
  if (action == 0):
    children.append(id[findValueIndex(form,s_word)])
    stack = np.delete(stack,-1)
    stack_word = np.delete(stack_word,-1)
    head[findValueIndex(form,s_word)] = id[findValueIndex(form,b_word)]
    action_name = 'Left Arc'

  # Right arc
  elif (action == 1):  #if s is the father of b
    children.append(id[findValueIndex(form,b_word)])
    stack = np.append(stack,b)
    stack_word = np.append(stack_word,b_word)
    buffer = np.delete(buffer,0)
    buffer_word = np.delete(buffer_word,0)
    head[findValueIndex(form,b_word)] = id[findValueIndex(form,s_word)]
    action_name = 'Right Arc'

  # Reduce
  elif (action == 2):
    stack = np.delete(stack,-1)
    stack_word = np.delete(stack_word,-1)
    action_name = 'Reduce'
  
  # Shift
  elif (action == 3):
    stack = np.append(stack,b)
    buffer = np.delete(buffer,0)
    action_name = 'Shift'

    stack_word = np.append(stack_word,b_word)
    buffer_word = np.delete(buffer_word,0)
  

  print(stack_word," | ",buffer_word," | ",action_name)
  (action,pred_deprel) = predTestValues(stack,buffer,children,model)

  # stack_vector = stackToVector(prev_stack,stack_spaces)
  # buffer_vector = bufferToVector(prev_buffer,buffer_spaces)
  # stack_upos_vector = stackToVector(prev_upos_stack,stack_spaces)
  # buffer_upos_vector = bufferToVector(prev_upos_buffer,buffer_spaces)
  # action_set = np.append(action_set,[action],axis = 0)
  # deprel_set = np.append(deprel_set,[rel],axis = 0)

  # x_vector = np.array([stack_vector,buffer_vector,stack_upos_vector,buffer_upos_vector],dtype='object')

  # if (i==0):
  #   x_set = np.append([x_set],[x_vector],axis = 0)
  # else:
  #   x_set = np.append(x_set,[x_vector],axis = 0)

  # i = i + 1
  
  # #print(stack_vector," | ",buffer_vector," | ",action," | ",rel," | ",stack_upos_vector," | ",buffer_upos_vector)

  # # Reduce and Done

  # if (len(buffer)==0):
  #   action = 4
  #   rel = 'None'
  #   stack_vector = stackToVector(stack,stack_spaces)
  #   buffer_vector = bufferToVector(buffer,buffer_spaces)
  #   stack_upos_vector = stackToVector(stack_upos,stack_spaces)
  #   buffer_upos_vector = bufferToVector(buffer_upos,buffer_spaces)
  #   action_set = np.append(action_set,[action],axis = 0)
  #   deprel_set = np.append(deprel_set,[rel],axis = 0)

  #   x_vector = np.array([stack_vector,buffer_vector,stack_upos_vector,buffer_upos_vector],dtype='object')
  #   x_set = np.append(x_set,[x_vector],axis = 0)

  #   #print(stack_vector," | ",buffer_vector," | ",action," | ",rel," | ",stack_upos_vector," | ",buffer_upos_vector)

  #   while (len(stack)>1):
  #     stack = np.delete(stack,-1)
  #     stack_upos = np.delete(stack_upos,-1)
  #     if (len(stack)>1):
  #       action = 4
  #       rel = 'None'
  #     elif(len(stack)==1):
  #       action = 5
  #       rel = 'None'
  #     stack_vector = stackToVector(stack,stack_spaces)
  #     buffer_vector = bufferToVector(buffer,buffer_spaces)
  #     stack_upos_vector = stackToVector(stack_upos,stack_spaces)
  #     buffer_upos_vector = bufferToVector(buffer_upos,buffer_spaces)
  #     action_set = np.append(action_set,[action],axis = 0)
  #     deprel_set = np.append(deprel_set,[rel],axis = 0)

  #     x_vector = np.array([stack_vector,buffer_vector,stack_upos_vector,buffer_upos_vector],dtype='object')
  #     x_set = np.append(x_set,[x_vector],axis = 0)

      #print(stack_vector," | ",buffer_vector," | ",action," | ",rel," | ",stack_upos_vector," | ",buffer_upos_vector)


NameError: ignored

In [ ]:
stack = tokenizer.texts_to_sequences(['root'])[0]
stackToVector(stack,4)

buffer = tokenizer.texts_to_sequences(['I love my car!'])[0]
bufferToVector(buffer,2)

array([35, 425], dtype=object)

In [ ]:
frase = 'I love my car too much'
buffer = tokenizer.texts_to_sequences([frase])[0]
lista = [*range(1, len(buffer)+1)]
lista

[1, 2, 3, 4, 5, 6]

In [ ]:
findValueIndex(buffer,'I')

IndexError: ignored

In [ ]:
tokenizer.sequences_to_texts(tokenizer.texts_to_sequences([frase]))

['i love my <OOV> too much']

In [ ]:
x_test_buffer

array([[2075.,   14., 2076., 3119.,    4.],
       [  14., 2076., 3119.,    4.,    0.],
       [2076., 3119.,    4.,    0.,    0.],
       ...,
       [   0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.]], dtype=float32)

In [ ]:
stack = 'root I'
buffer = 'love my car too much'

stack = tokenizer.texts_to_sequences([stack])[0]
buffer = tokenizer.texts_to_sequences([buffer])[0]

stack_vector = np.array([stackToVector(stack,3)],dtype=np.float32)
buffer_vector = np.array([bufferToVector(buffer,5)],dtype=np.float32)

pred_vector = model.predict([stack_vector,buffer_vector],verbose=0)[0][0]
print(pred_vector,pred_vector.argsort()[-(position+1)])

[0.11696301 0.28709835 0.02388444 0.57205415] 3


In [ ]:
print(stack_vector,buffer_vector)

[[3103.   35.  425.]] [[136.   1. 238. 154.   0.]]


In [ ]:
sentence = 'I love my car too much'
predNewValues(5,7,'I','love car too much',[],model)

(3, [])

In [ ]:
x_test_stack[0]
x_test_buffer[0]

array([2075.,   14., 2076., 3119.,    4.], dtype=float32)

In [ ]:
predTestValues(x_test_stack[0],x_test_buffer[0],[children],model)

(3, [[]])

NameError: ignored

In [ ]:
stack_vector = np.array([stackToVector(stack,3)],dtype=np.float32)
stack_vector

array([[   0., 3103.,   35.]], dtype=float32)

In [ ]:
def predNewValues(stack_len,buffer_len,stack,buffer,children,model):
  stack = tokenizer.texts_to_sequences([stack])[0]
  buffer = tokenizer.texts_to_sequences([buffer])[0]

  stack_vector = np.array([stackToVector(stack,stack_len)],dtype=np.float32)
  buffer_vector = np.array([bufferToVector(buffer,buffer_len)],dtype=np.float32)

  id_stack = [0]
  id_buffer = [*range(1, len(buffer)+1)]

  pred_action = model.predict([stack_vector,buffer_vector],verbose=0)[0][0]
  pred_deprel = model.predict([stack_vector,buffer_vector],verbose=0)[1][0]

  for position in range(4):
    action = pred_action.argsort()[-(position+1)]
    if action == 3: #Shift
      break
    if action == 0: #Left
      if (stack[-1] != tokenizer.texts_to_sequences(['root'])[0][0] and id_stack[-1] not in children):
        children.append(id_stack[-1])
        break
    if action == 1: #Right
      if (id_buffer[0] not in children):
        children.append(id_buffer[0])
        break
    if action == 2: #Reduce
      if (id_stack[-1] in children):
        break

  return (action,deprel)

def predTestValues(stack,buffer,children,model):
  stack_len = len(stack)
  buffer_len = len(buffer)

  stack_vector = np.array([stackToVector(stack,stack_len)],dtype=np.float32)
  buffer_vector = np.array([bufferToVector(buffer,buffer_len)],dtype=np.float32)


  id_stack = [0]
  id_buffer = [*range(1, len(buffer)+1)]

  pred_action = model.predict([stack_vector,buffer_vector],verbose=0)[0][0]
  deprel = np.argmax(model.predict([stack_vector,buffer_vector],verbose=0)[1][0])

  for position in range(4):
    action = pred_action.argsort()[-(position+1)]
    if action == 3: #Shift
      break
    if action == 0: #Left
      if (stack[-1] != tokenizer.texts_to_sequences(['root'])[0][0] and id_stack[-1] not in children):
        children.append(id_stack[-1])
        break
    if action == 1: #Right
      if (id_buffer[0] not in children):
        children.append(id_buffer[0])
        break
    if action == 2: #Reduce
      if (id_stack[-1] in children):
        break

  return (action,deprel)

In [ ]:
np.argmax(model.predict([stack_vector,buffer_vector],verbose=0)[1][0])

0

In [ ]:
# 0 : left
# 1 : right
# 2 : reduce
# 3 : shift

array([0.24242209, 0.21900505, 0.1537131 , 0.38485977], dtype=float32)

In [ ]:
np.array([stackToVector(stack,3)],dtype=np.float32)

array([[   0.,    0., 3103.]], dtype=float32)

In [ ]:
x_test_stack[:1]

array([[   0.,    0., 3103.]], dtype=float32)

In [ ]:
print("Generate a prediction")
stack_instance = np.array([stackToVector(stack,3)],dtype=np.float32)
buffer_instance = np.array([bufferToVector(buffer,5)],dtype=np.float32)
prediction = model.predict([stack_instance,buffer_instance])
prediction

Generate a prediction
1/1 [==============================] - 0s 22ms/step


[array([[3.1695467e-06, 1.3383089e-01, 2.7754673e-01, 5.8784753e-01,
         5.8345165e-04, 1.8813432e-04]], dtype=float32),
 array([[6.12027943e-01, 9.45203006e-03, 8.62767280e-04, 7.43075507e-04,
         2.53867283e-02, 2.28983350e-02, 1.40041753e-03, 1.22275958e-02,
         3.60415946e-03, 5.68776913e-02, 8.89895111e-03, 4.24856879e-03,
         8.36162886e-04, 2.56593782e-03, 8.73758085e-03, 8.95235222e-03,
         5.54547878e-04, 1.36726076e-05, 2.71545199e-04, 4.84252535e-02,
         1.08368564e-04, 5.55922998e-06, 2.01427168e-03, 4.31263959e-03,
         5.27694123e-03, 5.48626354e-04, 2.16381814e-04, 1.38395969e-02,
         2.15373784e-02, 5.31694423e-05, 4.03513154e-03, 3.40379030e-02,
         1.59823685e-03, 4.24124533e-03, 2.53195073e-02, 7.59100541e-03,
         8.80882726e-06, 1.30132776e-05, 2.95171943e-02, 1.11557310e-02,
         5.58395125e-03]], dtype=float32)]

In [ ]:
number2action[np.argmax(prediction[0][0])]

'Shift'

In [ ]:
def new_oracle(action,stack,buffer,stack_spaces,buffer_spaces):

  # re-write variables including root at the begining
  form = np.concatenate((['root'],text))
  id = np.concatenate(([int(0)],text_id))
  head = np.concatenate(([int(0)],text_head))
  deprel = np.concatenate((['root'],text_deprel))
  upos = np.concatenate(([int(0)],text_upos))

  # Creating the stack and buffer
  stack = np.array(['root'])
  buffer = text

  stack_upos = np.array([nupos+1])
  buffer_upos = text_upos

  target_spaces = 2 # action and deprel

  # Sets
  x_set = np.array([np.zeros(stack_spaces),np.zeros(buffer_spaces),np.zeros(stack_spaces),np.zeros(buffer_spaces)],dtype=object)
  action_set = [np.array(0)]
  deprel_set = [np.array(0)]

  # vector = np.array(
  #     [stackToVector(['1','1','1','1'],stack_spaces),
  #     bufferToVector(['1','1','1','1'],buffer_spaces),
  #     stackToVector(['1','1','1','1'],stack_spaces),
  #     bufferToVector(['1','1','1','1'],buffer_spaces)],
  #     dtype=object)

  # x_set = np.append([x_set],[vector],axis = 0)

  i = 0
  while len(buffer) > 0:
    s = stack[-1] #setting attention in the last element on stack
    b = buffer[0] #setting attention in the first element on buffer

    prev_stack = stack
    prev_buffer = buffer

    prev_upos_stack = stack_upos
    prev_upos_buffer = buffer_upos

    #Checking Right Arc
    # finValueIndex search the text of the current value and it returns the position
    
    if (head[findValueIndex(form,b)] == id[findValueIndex(form,s)]):  #if s is the father of b
      action = 1
      rel = deprel[findValueIndex(form,b)]
      stack = np.append(stack,b)
      buffer = np.delete(buffer,0)

      stack_upos = np.append(stack_upos,upos[findValueIndex(form,b)])
      buffer_upos = np.delete(buffer_upos,0)
    
    elif (head[findValueIndex(form,s)] == id[findValueIndex(form,b)]): # if b is the father of s
      action = 2
      rel = deprel[findValueIndex(form,s)]
      stack = np.delete(stack,-1)

      stack_upos = np.delete(stack_upos,-1)

    else: # if there is no relationship
      action = 3
      rel = 'None'
      stack = np.append(stack,b)
      buffer = np.delete(buffer,0)
      
      stack_upos = np.append(stack_upos,upos[findValueIndex(form,b)])
      buffer_upos = np.delete(buffer_upos,0)

    stack_vector = stackToVector(prev_stack,stack_spaces)
    buffer_vector = bufferToVector(prev_buffer,buffer_spaces)
    stack_upos_vector = stackToVector(prev_upos_stack,stack_spaces)
    buffer_upos_vector = bufferToVector(prev_upos_buffer,buffer_spaces)
    action_set = np.append(action_set,[action],axis = 0)
    deprel_set = np.append(deprel_set,[rel],axis = 0)

    x_vector = np.array([stack_vector,buffer_vector,stack_upos_vector,buffer_upos_vector],dtype='object')

    if (i==0):
      x_set = np.append([x_set],[x_vector],axis = 0)
    else:
      x_set = np.append(x_set,[x_vector],axis = 0)

    i = i + 1
    
    #print(stack_vector," | ",buffer_vector," | ",action," | ",rel," | ",stack_upos_vector," | ",buffer_upos_vector)

    # Reduce and Done

    if (len(buffer)==0):
      action = 4
      rel = 'None'
      stack_vector = stackToVector(stack,stack_spaces)
      buffer_vector = bufferToVector(buffer,buffer_spaces)
      stack_upos_vector = stackToVector(stack_upos,stack_spaces)
      buffer_upos_vector = bufferToVector(buffer_upos,buffer_spaces)
      action_set = np.append(action_set,[action],axis = 0)
      deprel_set = np.append(deprel_set,[rel],axis = 0)

      x_vector = np.array([stack_vector,buffer_vector,stack_upos_vector,buffer_upos_vector],dtype='object')
      x_set = np.append(x_set,[x_vector],axis = 0)

      #print(stack_vector," | ",buffer_vector," | ",action," | ",rel," | ",stack_upos_vector," | ",buffer_upos_vector)

      while (len(stack)>1):
        stack = np.delete(stack,-1)
        stack_upos = np.delete(stack_upos,-1)
        if (len(stack)>1):
          action = 4
          rel = 'None'
        elif(len(stack)==1):
          action = 5
          rel = 'None'
        stack_vector = stackToVector(stack,stack_spaces)
        buffer_vector = bufferToVector(buffer,buffer_spaces)
        stack_upos_vector = stackToVector(stack_upos,stack_spaces)
        buffer_upos_vector = bufferToVector(buffer_upos,buffer_spaces)
        action_set = np.append(action_set,[action],axis = 0)
        deprel_set = np.append(deprel_set,[rel],axis = 0)

        x_vector = np.array([stack_vector,buffer_vector,stack_upos_vector,buffer_upos_vector],dtype='object')
        x_set = np.append(x_set,[x_vector],axis = 0)

        #print(stack_vector," | ",buffer_vector," | ",action," | ",rel," | ",stack_upos_vector," | ",buffer_upos_vector)
  return (x_set[1:],action_set[1:],deprel_set[1:])

In [ ]:
number2action = {
  1: 'Right Arc', 
  2: 'Left Arc', 
  3: 'Shift', 
  4: 'Reduce', 
  5: 'Done'
}

number2deprel_test = {
  0: 'None',
  1: 'acl',
  2: 'acl:relcl',
  3: 'advcl',
  4: 'advmod',
  5: 'amod',
  6: 'appos',
  7: 'aux',
  8: 'aux:pass',
  9: 'case',
  10: 'cc',
  11: 'ccomp',
  12: 'compound',
  13: 'compound:prt',
  14: 'conj',
  15: 'cop',
  16: 'det',
  17: 'det:predet',
  18: 'expl',
  19: 'fixed',
  20: 'flat',
  21: 'flat:foreign',
  22: 'iobj',
  23: 'mark',
  24: 'nmod',
  25: 'nmod:poss',
  26: 'nsubj',
  27: 'nsubj:pass',
  28: 'nummod',
  29: 'obj',
  30: 'obl',
  31: 'punct',
  32: 'root',
  33: 'xcomp'
}

In [ ]:
# Testing different batch_sizes

pivot_name = 'batch_size'
#pivot = [32,64,128,256,512,1024,2048,4096]
pivot = [1024,2048,4096]

# Setting values
stack_len = 3
buffer_len = 3
arquitecture = 'A'
stopper = 'val_Deprel_Output_accuracy'
patience = 3
batch_size = 1024
epochs = 10
optimizer = 'adam'
learning_rate = 0.1
embedd_output = 50

(df_res_batch_size,models_batch_size) = saveModelAData(pivot_name,pivot,
                   x_train_stack,x_train_buffer,action_train, deprel_train,
                   x_val_stack,x_val_buffer,action_val,deprel_val,
                   x_test_stack,x_test_buffer,action_test,deprel_test,
                   stack_len,buffer_len,
                   stopper,patience,
                   batch_size,epochs,
                   optimizer,learning_rate,
                   embedd_output=50)

Starting execution where  batch_size  varies, now with the value(s)  1024 .
Starting execution where  batch_size  varies, now with the value(s)  2048 .
Starting execution where  batch_size  varies, now with the value(s)  4096 .


In [ ]:
path = 'model_testing/batch_size/'

!mkdir -p {path}

df_res_batch_size.to_csv('model_testing/batch_size/batch_size_models.csv', index=False)

In [ ]:
# Testing different early stop configuration

pivot_name = 'early_stop'
pivot = [
('loss',3),
('loss',5),
('loss',7),
('Action_Output_loss',3),
('Action_Output_loss',5),
('Action_Output_loss',7),
('Deprel_Output_loss',3),
('Deprel_Output_loss',5),
('Deprel_Output_loss',7),
('Action_Output_accuracy',3),
('Action_Output_accuracy',5),
('Action_Output_accuracy',7),
('Deprel_Output_accuracy',3),
('Deprel_Output_accuracy',5),
('Deprel_Output_accuracy',7),
('val_loss',3),
('val_loss',5),
('val_loss',7),
('val_Action_Output_loss',3),
('val_Action_Output_loss',5),
('val_Action_Output_loss',7),
('val_Deprel_Output_loss',3),
('val_Deprel_Output_loss',5),
('val_Deprel_Output_loss',7),
('val_Action_Output_accuracy',3),
('val_Action_Output_accuracy',5),
('val_Action_Output_accuracy',7),
('val_Deprel_Output_accuracy',3),
('val_Deprel_Output_accuracy',5),
('val_Deprel_Output_accuracy',7)
]

# Setting values
stack_len = 3
buffer_len = 3
arquitecture = 'A'
stopper = 'val_Deprel_Output_loss'
patience = 3
batch_size = 1024
epochs = 10
optimizer = 'adam'
learning_rate = 0.1
embedd_output = 50

(df_res_earlystop,models_earlystop) = saveModelAData(pivot_name,pivot,
                   x_train_stack,x_train_buffer,action_train, deprel_train,
                   x_val_stack,x_val_buffer,action_val,deprel_val,
                   x_test_stack,x_test_buffer,action_test,deprel_test,
                   stack_len,buffer_len,
                   stopper,patience,
                   batch_size,epochs,
                   optimizer,learning_rate,
                   embedd_output=50)

Starting execution where  early_stop  varies, now with the value(s)  ('loss', 3) .
Starting execution where  early_stop  varies, now with the value(s)  ('loss', 5) .
Starting execution where  early_stop  varies, now with the value(s)  ('loss', 7) .
Starting execution where  early_stop  varies, now with the value(s)  ('Action_Output_loss', 3) .
Starting execution where  early_stop  varies, now with the value(s)  ('Action_Output_loss', 5) .
Starting execution where  early_stop  varies, now with the value(s)  ('Action_Output_loss', 7) .
Starting execution where  early_stop  varies, now with the value(s)  ('Deprel_Output_loss', 3) .
Starting execution where  early_stop  varies, now with the value(s)  ('Deprel_Output_loss', 5) .
Starting execution where  early_stop  varies, now with the value(s)  ('Deprel_Output_loss', 7) .
Starting execution where  early_stop  varies, now with the value(s)  ('Action_Output_accuracy', 3) .
Starting execution where  early_stop  varies, now with the value(s) 

In [ ]:
df_res_earlystop

,arquitecture,stack,buffer,action_accuracy,deprel_accuracy,action_loss,deprel_loss,batch_size,epochs,optimizer,learning_rate,embedd_output,early_stop_set,time
0,A,3,3,0.711409,0.707032,0.554237,2.161890,1024,10,adam,0.1,50,loss,64.576141
1,A,3,3,0.753720,0.703385,0.500218,2.009624,1024,10,adam,0.1,50,loss,63.566917
2,A,3,3,0.809163,0.708491,0.449651,2.179028,1024,10,adam,0.1,50,loss,62.367026
3,A,3,3,0.713014,0.684856,0.603775,2.177422,1024,10,adam,0.1,50,Action_Output_loss,58.097581
4,A,3,3,0.775751,0.696236,0.464987,2.121894,1024,10,adam,0.1,50,Action_Output_loss,63.113043
5,A,3,3,0.707032,0.707032,0.654016,1.709301,1024,10,adam,0.1,50,Action_Output_loss,63.177822
6,A,3,3,0.707032,0.707032,0.654839,1.749806,1024,10,adam,0.1,50,Deprel_Output_loss,52.418084
7,A,3,3,0.790925,0.682083,0.447916,1.761977,1024,10,adam,0.1,50,Deprel_Output_loss,62.323539
8,A,3,3,0.712577,0.690254,0.579835,1.900675,1024,10,adam,0.1,50,Deprel_Output_loss,63.229553
9,A,3,3,0.787861,0.706449,0.490878,1.621736,1024,10,adam,0.1,50,Action_Output_accuracy,52.033739


In [ ]:
# Testing different optimizers

pivot_name = 'optimizer'
pivot = [
    ('adam' , 0.001),
    ('adam' , 0.01),
    ('adam' , 0.1),
    ('sgd' , 0.001),
    ('sgd' , 0.01),
    ('sgd' , 0.1),
    ('rmsprop' , 0.001),
    ('rmsprop' , 0.01),
    ('rmsprop' , 0.1),
    ('adadelta' , 0.001),
    ('adadelta' , 0.01),
    ('adadelta' , 0.1),
    ('adagrad' , 0.001),
    ('adagrad' , 0.01),
    ('adagrad' , 0.1),
    ('adamax' , 0.001),
    ('adamax' , 0.01),
    ('adamax' , 0.1),
    ('nadam' , 0.001),
    ('nadam' , 0.01),
    ('nadam' , 0.1),
    ('ftrl' , 0.001),
    ('ftrl' , 0.01),
    ('ftrl' , 0.1)
]

# Setting values
stack_len = 3
buffer_len = 3
arquitecture = 'A'
stopper = 'val_Deprel_Output_loss'
patience = 3
batch_size = 1024
epochs = 10
optimizer = 'adam'
learning_rate = 0.01
embedd_output = 50

(df_res_optimizers,models_optimizers) = saveModelAData(pivot_name,pivot,
                   x_train_stack,x_train_buffer,action_train, deprel_train,
                   x_val_stack,x_val_buffer,action_val,deprel_val,
                   x_test_stack,x_test_buffer,action_test,deprel_test,
                   stack_len,buffer_len,
                   stopper,patience,
                   batch_size,epochs,
                   optimizer,learning_rate,
                   embedd_output=50)

In [ ]:
res_df,models = result_df
result_df

(   arquitecture  stack  buffer  action_accuracy  deprel_accuracy  action_loss  \
 0             A      3       3         0.684710         0.707032     0.735670   
 1             A      3       3         0.684856         0.707032     0.656319   
 2             A      3       3         0.772542         0.707032     0.477825   
 3             A      3       3         0.398162         0.707032     1.372104   
 4             A      3       3         0.398162         0.707032     1.365990   
 5             A      3       3         0.398162         0.707032     1.212451   
 6             A      3       3         0.667493         0.707032     0.855332   
 7             A      3       3         0.705282         0.707032     0.642703   
 8             A      3       3         0.666472         0.707032     0.677105   
 9             A      3       3         0.286694         0.000000     1.726953   
 10            A      3       3         0.000000         0.000000     1.899116   
 11            A

In [ ]:
res_df.iloc[:,3:]

,action_accuracy,deprel_accuracy,action_loss,deprel_loss,batch_size,epochs,optimizer,learning_rate,embedd_output,early_stop_set,time
0,0.684710,0.707032,0.735670,1.766074,1024,10,adam,0.001,50,val_Deprel_Output_loss,42.021977
1,0.684856,0.707032,0.656319,1.697979,1024,10,adam,0.010,50,val_Deprel_Output_loss,33.284477
2,0.772542,0.707032,0.477825,1.631779,1024,10,adam,0.100,50,val_Deprel_Output_loss,33.566783
3,0.398162,0.707032,1.372104,1.660246,1024,10,sgd,0.001,50,val_Deprel_Output_loss,92.364595
4,0.398162,0.707032,1.365990,1.750521,1024,10,sgd,0.010,50,val_Deprel_Output_loss,51.367592
5,0.398162,0.707032,1.212451,1.656508,1024,10,sgd,0.100,50,val_Deprel_Output_loss,49.190301
6,0.667493,0.707032,0.855332,1.688933,1024,10,rmsprop,0.001,50,val_Deprel_Output_loss,34.432504
7,0.705282,0.707032,0.642703,1.732643,1024,10,rmsprop,0.010,50,val_Deprel_Output_loss,52.447595
8,0.666472,0.707032,0.677105,1.708530,1024,10,rmsprop,0.100,50,val_Deprel_Output_loss,52.370254
9,0.286694,0.000000,1.726953,3.543139,1024,10,adadelta,0.001,50,val_Deprel_Output_loss,92.795181


In [ ]:
path = 'model_testing/early_stop/'

!mkdir -p {path}

In [ ]:
df_res_earlystop.to_csv('model_testing/early_stop/early_stop_models.csv', index=False)

**restore_best_weights:** Whether to restore model weights from the epoch with the best value of the monitored quantity. If False, the model weights obtained at the last step of training are used